In [2]:
import datetime
import sys

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

import pandas as pd
from pandas_datareader import data
from itertools import islice

from scipy.signal import savgol_filter
import statsmodels
import statsmodels.api as sm
import scipy.stats as stats
import scipy.optimize as optimize
import numpy as np
#from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
#For the usual plots!
#matplotlib inline
#Makes graphs in jupyter interactive!
%matplotlib widget

#Removing warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
plt.close('all') 

In [4]:
#########                        Inputs
    #Tickers used
tickers = ['SPY', 'NVDA', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['NVDA']
    #Data timeframe
startd = datetime.datetime(2016, 1, 1) 
endd_2 = datetime.datetime(2020, 1, 1)
    #Inputs BB
bb1input = 'Open'
bb1period = 50
bb1multiplier = 1
bb2input = 'Open'
bb2period = 50
bb2multiplier = 1.5
bb3input = 'Open'
bb3period = 20
bb3multiplier = 1
bb4input = 'Open'
bb4period = 50
bb4multiplier = 1
    #Inputs ATR
atr1period = 10
atr1multiplier = 1.25
atr1plateau = 1
atr2period = 50
atr2multiplier = 0.75
atr2plateau = 2
    #Inputs Decycler
d1input = 'Open'
d1period = 50
d1upper = 1.01
d1lower= 0.99
d2input = 'Open'
d2period = 25
d2upper = 1.01
d2lower= 0.99
    #Inputs SMA
sma1input = 'Open'
sma1period = 5
sma2input = 'Open'
sma2period = 10
######### 

######################           Functions
def stock_data(startd, endd, tickers=tickers):
    df = data.DataReader(tickers, 'yahoo', startd, endd).reset_index()
#df.set_index('Date', inplace=True)            <-- weggehaald zodat polyfit en backtest "i-1" werken
#uitrekenen van de daily return (cert)
    for ticker in tickers:
        df[('Return', ticker)] = df[('Close', ticker)].shift(1) - df[('Close', ticker)] 
    for ticker in tickers:
        df[('Percent_return', ticker)] = df[('Close', ticker)].pct_change()
    return df

def ehlers_simple_decycle(df, src, hpLength, t, name, upper, lower):
    #hp = highpass
    alphaArg = 2*np.pi / (hpLength*np.sqrt(2))
    alpha = 'Alpha_' + name
    hp = 'Hp_' + name
    decycler = 'Decycler_' + name
    decycleru = 'Decycleru_' + name
    decyclerl = 'Decyclerl_' + name
    df[(alpha, t)] = 0        
    df[(hp, t)] = 0   
    df[(decycler, t)] = 0 
    df[(decycleru, t)] = 0 
    df[(decyclerl, t)] = 0 
    for i in range(hpLength, len(df)):
        #Hp filter
        if np.cos(alphaArg) != 0:
            df.loc[i, (alpha, t)] = (np.cos(alphaArg)+np.sin(alphaArg)-1) / np.cos(alphaArg)
        else:
            df.loc[i, (alpha, t)] = df.loc[i-1, (alpha, t)]
        #Decycling::: src = the input!
        df.loc[i, (hp, t)] = np.square(1-(df.loc[i, (alpha, t)]/2)) * (src[i]-2 * src[i-1]+src[i-2]) + 2 * (1-df.loc[i, (alpha, t)]) * df.loc[i-1, (hp, t)] - np.square(1-df.loc[i, (alpha, t)]) * df.loc[i-2, (hp, t)]
        df.loc[i, (decycler, t)] = src[i] - df.loc[i, (hp, t)]
        df.loc[i, (decycleru, t)] = df.loc[i, (decycler, t)]*upper
        df.loc[i, (decyclerl, t)] = df.loc[i, (decycler, t)]*lower
    return df
    
def bolinger_bands(df, close, length, multiple):
    for i in range(len(df)):
        mean_bb = close.rolling(length).mean()
        std  = close.rolling(length).std()
        upper_bb = mean_bb + (std*multiple)
        lower_bb = mean_bb - (std*multiple)
        return mean_bb, upper_bb, lower_bb   
    
#Functions Average True Range#
def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(data, t , n, multiplier):
    high = data[('High', t)]
    low = data[('Low', t)]
    close = data[('Close', t)]
    data[('Tr0', t)] = abs(high - low)
    data[('Tr1', t)] = abs(high - close.shift())
    data[('Tr2', t)] = abs(low - close.shift())
    Tr = data[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    Atr = wwma(Tr, n)
    data[('Atr', t)] = Atr
#De Stoploss 2xATR implementatie:
    Stoploss = data[('Close', t)] - (data[('Atr', t)] * multiplier)
    return Stoploss
######################

#===~~~~====~~~~~~===~~~~====~~~~~~          Start             ===~~~~====~~~~~~===~~~~====~~~~~~ 
time1 = datetime.datetime.now().time()
print(f'1. Script Started timestamp =     {time1}')

############                       DATA
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
############

print(f'Data, amount of rows = {df.index.max()}')    #For checking if dataframe length stays consistent!


###~~~~~~~~~~~~~~~~~~~~~~~~            HAVE TO CLEAN THESE UP FOR REDUDANCY  <<<<<<<<<<<<<<<<<<<<<<------------------

######################             SELL 1
#BB1; Open, 50, 1
for t in tickers:
    df[('Mean_bb1', t)], df[('Upper_bb1', t)], df[('Lower_bb1', t)] = bolinger_bands(df, df[(bb1input, t)], bb1period, bb1multiplier)
    #Close > BBU
    df[('Strat_sell1_close_bb1u', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i, ('Close', t)] > df.loc[i, ('Upper_bb1', t)]: 
            df.loc[i, ('Strat_sell1_close_bb1u', t)] = 1
#ATR1; 10 periods, 3 peaks, 1.25 multiplier, Atrx
for t in tickers:
    df[('Stoploss_atr1', t)] = atr(df, t, atr1period, atr1multiplier)

    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss1', t)] = df.loc[x:i, ('Stoploss_atr1', t)].max()
        try:
            if df.loc[i, ('Stoploss1', t)] == df.loc[i-1, ('Stoploss1', t)]:
                x=i
        except:
            pass
        if df.loc[i, ('Stoploss1', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i:i+atr1plateau, ('Strat_sell1_atr1', t)] = 1
            next(islice(loop, atr1plateau-1, atr1plateau-1), None)
            x=i
        if df.loc[i, ('Stoploss1', t)] < df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_sell1_atr1', t)] = 0  
#Decycler 1; 50, Open, 1%;band
for t in tickers:
    df = ehlers_simple_decycle(df, df[(d1input, t)], d1period, t, 'd1', d1upper, d1lower)
    
#    for i in range(1, len(df)-1):
#        if df.loc[i, ('Decycler_d1', t)] >= df.loc[i-1, ('Decycler_d1', t)]: 
#            df.loc[i, ('Strat_sell1_d1m', t)] = 1   # Green
#        else:
#            df.loc[i, ('Strat_sell1_d1m', t)] = 0   # Red
    #D1U > Close
    for i in range(1, len(df)-1):
        if df.loc[i, ('Decycleru_d1', t)] >= df.loc[i, ('Close', t)]: 
            df.loc[i, ('Strat_sell1_close_d1u', t)] = 1 
        else:
            df.loc[i, ('Strat_sell1_close_d1u', t)] = 0 
    #D1 > BBU
    df[('Strat_sell1_bb1u_d1m', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i, ('Decycler_d1', t)] >= df.loc[i, ('Upper_bb1', t)]: 
            df.loc[i, ('Strat_sell1_bb1u_d1m', t)] = 1 

#SMA; Open, 5
for t in tickers:
    df[('SMA1', t)] = df[(sma1input, t)].rolling(sma1period).mean()
    
    #Sma5 c= d1bands
    df[('Strat_sell1_d1b_sma1', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i, ('SMA1', t)] >= df.loc[i, ('Decyclerl_d1', t)] and df.loc[i, ('SMA1', t)] <= df.loc[i, ('Decycleru_d1', t)]:
            df.loc[i, ('Strat_sell1_d1b_sma1', t)] = 1

###Bringing all together -- Sell 1
for t in tickers:
    df[('Sell1_signal', t)] = 0
    for i in range(len(df)-1):
        if df.loc[i, ('Strat_sell1_atr1', t)] == 1:
            if df.loc[i, ('Strat_sell1_d1b_sma1', t)] == 1:
                if df.loc[i, ('Strat_sell1_bb1u_d1m', t)] == 1:
                    if df.loc[i, ('Strat_sell1_close_d1u', t)] == 1:
                        df.loc[i, ('Sell1_signal', t)] = 1
######################

############################################        SELL 2
#BB2; Open, 50, 1.5
for t in tickers:
    df[('Mean_bb2', t)], df[('Upper_bb2', t)], df[('Lower_bb2', t)] = bolinger_bands(df, df[(bb2input, t)], bb2period, bb2multiplier)

    for i in range(1, len(df)-1):
        if df.loc[i, ('Close', t)] > df.loc[i, ('Upper_bb2', t)]:
            df.loc[i, ('Strat_sell2_bb2u', t)] = 1
        if df.loc[i, ('Close', t)] <= df.loc[i, ('Upper_bb2', t)]:
            df.loc[i, ('Strat_sell2_bb2u', t)] = 0 
#ATR2; 50 periods, 2 peaks, 0.75 multiplier, Atrx
for t in tickers:
    df[('Stoploss_atr2', t)] = atr(df, t, atr2period, atr2multiplier)

    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss2', t)] = df.loc[x:i, ('Stoploss_atr2', t)].max()
        try:
            if df.loc[i, ('Stoploss2', t)] == df.loc[i-1, ('Stoploss2', t)]:
                x=i
        except:
            pass
        if df.loc[i, ('Stoploss2', t)] >= df.loc[i, ('Close', t)]:
            df.loc[i:i+atr2plateau, ('Strat_sell2_atr2', t)] = 1
            next(islice(loop, atr2plateau-1, atr2plateau-1), None)
            x=i
        if df.loc[i, ('Stoploss2', t)] < df.loc[i, ('Close', t)]:
            df.loc[i, ('Strat_sell2_atr2', t)] = 0    
#Decycler 2; 50, Open, 1%;band
for t in tickers:
    df = ehlers_simple_decycle(df, df[(d2input, t)], d2period, t, 'd2', d2upper, d2lower)
    
#    for i in range(1, len(df)-1):
#        if df.loc[i, ('Decycler_d2', t)] >= df.loc[i-1, ('Decycler_d2', t)]: 
#            df.loc[i, ('Strat_sell2_d2m', t)] = 1   # Green
#        else:
#            df.loc[i, ('Strat_sell2_d2m', t)] = 0   # Red
    #D2 > Close
    for i in range(1, len(df)-1):
        if df.loc[i, ('Decycleru_d2', t)] >= df.loc[i, ('Close', t)]: 
            df.loc[i, ('Strat_sell2_close_d2u', t)] = 1 
        else:
            df.loc[i, ('Strat_sell2_close_d2u', t)] = 0 
#Sma5
for t in tickers:
    #Sma5 c= d1bands
    for i in range(1, len(df)-1):
        if df.loc[i, ('SMA1', t)] >= df.loc[i, ('Decyclerl_d1', t)] and df.loc[i, ('SMA1', t)] <= df.loc[i, ('Decycleru_d1', t)]:
            df.loc[i, ('Strat_sell1_d1b_sma1', t)] = 1
        else:
            df.loc[i, ('Strat_sell1_d1b_sma1', t)] = 0
###Bringing all together -- Sell 2
for t in tickers:
    df[('Sell2_signal', t)] = 0
    for i in range(len(df)-1):
        if df.loc[i, ('Strat_sell2_atr2', t)] == 1:
            if df.loc[i, ('Strat_sell2_close_d2u', t)] == 1:
                if df.loc[i, ('Strat_sell2_bb2u', t)] == 1:
                    if df.loc[i, ('Strat_sell1_d1b_sma1', t)] == 1:
                        df.loc[i, ('Sell2_signal', t)] = 1
############################################

#############################################################      SELL 3
#BB3; Open, 20, 1
for t in tickers:
    df[('Mean_bb3', t)], df[('Upper_bb3', t)], df[('Lower_bb3', t)] = bolinger_bands(df, df[(bb3input, t)], bb3period, bb3multiplier)

    for i in range(1, len(df)-1):
        if df.loc[i, ('Close', t)] <= df.loc[i, ('Mean_bb3', t)]:
            df.loc[i, ('Strat_sell3_close_bb3m', t)] = 1
        if df.loc[i, ('Close', t)] > df.loc[i, ('Mean_bb3', t)]:
            df.loc[i, ('Strat_sell3_close_bb3m', t)] = 0 
#BB4; == BB1; Open, 50, 1
for t in tickers:
    df[('Mean_bb4', t)], df[('Upper_bb4', t)], df[('Lower_bb4', t)] = df[('Mean_bb1', t)], df[('Upper_bb1', t)], df[('Lower_bb1', t)]
#SMA; Open, 10
for t in tickers:
    df[('SMA2', t)] = df[(sma2input, t)].rolling(sma2period).mean()
    
    df[('Sell32_signal', t)] = 0
    df[('Sell31_signal', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i, ('SMA1', t)] and df.loc[i, ('SMA2', t)] > df.loc[i, ('Close', t)]:
            if df.loc[i, ('SMA1', t)] and df.loc[i, ('SMA2', t)] > df.loc[i, ('Upper_bb4', t)]:
                if df.loc[i, ('SMA2', t)] > df.loc[i, ('SMA1', t)]:
                    df.loc[i, ('Sell32_signal', t)] = 1
    
    for i in range(1, len(df)-1):     
        #if df.loc[i, ('Close', t)] <= df.loc[i, ('Mean_bb4', t)] and df.loc[i, ('Close', t)] <= df.loc[i, ('Mean_bb3', t)] and (df.loc[i-6, ('Close', t)] >= df.loc[i-6, ('Upper_bb4',t)] or df.loc[i-5, ('Close', t)] >= df.loc[i-5, ('Upper_bb4',t)]):
         #   df.loc[i, ('Sell3_signal', t)] = 1
        if df.loc[i, ('Close', t)] <= df.loc[i, ('Upper_bb4', t)] and df.loc[i, ('Upper_bb4', t)] > df.loc[i, ('Mean_bb3', t)] and (df.loc[i-6, ('Close', t)] >= df.loc[i-6, ('Upper_bb4',t)] or df.loc[i-5, ('Close', t)] >= df.loc[i-5, ('Upper_bb4',t)]):
            df.loc[i, ('Sell31_signal', t)] = 1
            
#############################################################    

#############    NOTITIE       -Vind iets voor de dode momenten, side ways trend! -> of niks doen??  
#                                    -> Wanneer BB4M ⊆ BB3 Upperband Only selling allowed? Different selling rule?
#                              -Verkoop regels Wanneer je in een down trend koopt!  <<-- Verdient onderscheid!
#                                       ~~~~Misschien een lange decycler gebruiken

#####################################################################################     BUY STRATS 4
####Buy 1; Rising trend

#Maybe only use when we want to buy back, thus a strict rising only filter!  <<<No succes yet.....
#Try to find a way to exclude sideways movement!
for t in tickers:
    df[('Buy1_signal', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i, ('Mean_bb4', t)] >= df.loc[i-1, ('Mean_bb4', t)]:
        #if df.loc[i, ('SMA1', t)] > df.loc[i, ('SMA2', t)]:  
        #if df.loc[i, ('Close', t)] > df.loc[i, ('Upper_bb3', t)] and df.loc[i-1, ('Close', t)] <= df.loc[i-1, ('Upper_bb3', t)]:
             #   df.loc[i, ('Buy1_signal', t)] = 1
            if df.loc[i, ('Close', t)] > df.loc[i, ('Mean_bb3', t)] and df.loc[i-1, ('Close', t)] <= df.loc[i-1, ('Mean_bb3', t)]:
                df.loc[i, ('Buy1_signal', t)] = 1
            #if df.loc[i, ('Close', t)] > df.loc[i, ('Lower_bb3', t)] and df.loc[i-1, ('Close', t)] <= df.loc[i-1, ('Lower_bb3', t)]:
             #   df.loc[i, ('Buy1_signal', t)] = 1

####Buy 2; Long drop 

#MAYBE THIS ONE IS ENOUGH????
#Check for potential filters for the 2 buy moments, dont want it triggered at a sideways trend!
for t in tickers:
    df[('Buy2_signal', t)] = 0
    for i in range(1, len(df)-1):
        #if df.loc[i, ('Mean_bb4', t)] <= df.loc[i-1, ('Mean_bb4', t)]:
        if df.loc[i, ('SMA1', t)] > df.loc[i, ('Lower_bb4', t)] and df.loc[i-1, ('SMA1', t)] <= df.loc[i-1, ('Lower_bb4', t)]:
            df.loc[i, ('Buy2_signal', t)] = 1
        elif df.loc[i, ('SMA2', t)] >= df.loc[i, ('Mean_bb4', t)] and df.loc[i-1, ('SMA2', t)] < df.loc[i-1, ('Mean_bb4', t)]:
            df.loc[i, ('Buy2_signal', t)] = 1
                
##################################################################################### 
#===~~~~====~~~~~~===~~~~====~~~~~~          Half             ===~~~~====~~~~~~===~~~~====~~~~~~ 
time2 = datetime.datetime.now().time()
print(f'3. Script Halfway timestamp =   {time2}')
print(f'Data, amount of rows = {df.index.max()}')    #For checking if dataframe length stays consistent!

################################################        STRATEGIE
####Sell strategie
for t in tickers:
    df[('Strat_s', t)] = 0
#    for i in range(len(df)-1):
#        if df.loc[i, ('Strat_sma1_signal', t)] == 1:


####Buy Strategie
for t in tickers:
    df[('Strat_b', t)] = 0
#    for i in range(len(df)-1):
#        if df.loc[i, ('Strat_decycle_d1_signal', t)] == 1:

###Totale Strategie ------------------------------------------
for t in tickers:
    i = 0  #longest sma = 20!
    while i < len(df)-1:
        if df.loc[i, ('Strat_s', t)] == 1:
            while True:
                if df.loc[i, ('Strat_b', t)] == 0:
                    df.loc[i+1, ('Strat', t)] = 0
                    i+=1
                    if i == len(df)-1:
                        break
                if df.loc[i, ('Strat_b', t)] == 1:
                    df.loc[i+1, ('Strat', t)] = 1
                    i+=1
                    break               
        if df.loc[i, ('Strat_s', t)] == 0:
            df.loc[i+1, ('Strat', t)] = 1
            i+=1
            if i == len(df)-1:
                break
################################################

#===~~~~====~~~~~~===~~~~====~~~~~~          END             ===~~~~====~~~~~~===~~~~====~~~~~~ 
time2 = datetime.datetime.now().time()
print(f'2. Script Completed timestamp =   {time2}')
print(f'Data, amount of rows = {df.index.max()}')    #For checking if dataframe length stays consistent!

1. Script Started timestamp =     03:01:04.757504
Data, amount of rows = 1005
3. Script Halfway timestamp =   03:01:45.540342
Data, amount of rows = 1005
2. Script Completed timestamp =   03:01:46.994792
Data, amount of rows = 1005


In [ ]:
df[('Strat_return', 'Total')] = 0
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
    
    df[('Strat_return', t)] = df[('Strat', t)] * df[('Percent_return', t)]
    df[('Strat_return', 'Total')] += df[('Strat_return', t)].cumsum()
for t in tickers:
    fig, ax = plt.subplots(5, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5, 1.5, 0.5, 0.5], 'hspace':0.02})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)])
    ax[1].plot(df[('Strat', t)])
    ax[2].plot(df[('Percent_return', t)].cumsum(), linewidth=1)
    ax[2].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
    ax[3].plot(df[('Strat_b', t)])
    ax[4].plot(df[('Strat_s', t)])
    r1=df[('Strat_return', t)].sum()
    r2=df[('Percent_return', t)].sum()
    print(f'Strat return {t} = {r1}')
    print(f'Hold return {t} = {r2}')

In [177]:
print(df.index.max())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

1005


Attributes       Date   Adj Close       Close        High         Low  \
Symbols                      NVDA        NVDA        NVDA        NVDA   
0          2016-01-04   31.672333   32.369999   32.580002   32.040001   
1          2016-01-05   32.181126   32.889999   33.439999   32.500000   
2          2016-01-06   30.850433   31.530001   32.500000   31.160000   
3          2016-01-07   29.627378   30.280001   30.950001   29.879999   
4          2016-01-08   28.991388   29.629999   30.700001   29.570000   
5          2016-01-11   29.040312   29.680000   29.889999   29.150000   
6          2016-01-12   29.529535   30.180000   30.620001   29.969999   
7          2016-01-13   28.629364   29.260000   30.610001   29.230000   
8          2016-01-14   28.052078   28.670000   29.000000   27.820000   
9          2016-01-15   26.525700   27.110001   27.850000   26.639999   
10         2016-01-19   26.740953   27.330000   28.450001   27.219999   
11         2016-01-20   26.887722   27.480000   27.830000   26.450001   
12         2016-01-21   27.200829   27.799999   28.230000   27.209999   
13         2016-01-22   27.836821   28.450001   28.639999   28.270000   
14         2016-01-25   27.807468   28.420000   28.770000   28.360001   
15         2016-01-26   28.081430   28.700001   28.820000   28.190001   
16         2016-01-27   27.748758   28.360001   28.750000   27.820000   
17         2016-01-28   27.445438   28.049999   28.690001   27.500000   
18         2016-01-29   28.658714   29.290001   29.350000   28.180000   
19         2016-02-01   28.668497   29.299999   29.450001   29.010000   
20         2016-02-02   27.455223   28.059999   29.150000   27.860001   
21         2016-02-03   27.592207   28.200001   28.450001   27.549999   
22         2016-02-04   27.601992   28.209999   28.850000   27.930000   
23         2016-02-05   25.860357   26.430000   28.330000   26.350000   
24         2016-02-08   24.676434   25.219999   26.160000   24.850000   
25         2016-02-09   24.940613   25.490000   26.549999   24.799999   
26         2016-02-10   24.881905   25.430000   26.389999   25.290001   
27         2016-02-11   24.754711   25.299999   25.559999   24.780001   
28         2016-02-12   25.175440   25.730000   25.780001   24.750000   
29         2016-02-16   26.408287   26.990000   27.030001   26.020000   
30         2016-02-17   27.063847   27.660000   27.920000   27.219999   
31         2016-02-18   29.392551   30.040001   30.980000   29.900000   
32         2016-02-19   29.783928   30.440001   30.709999   29.650000   
33         2016-02-22   30.840649   31.520000   31.870001   30.820000   
34         2016-02-23   30.918924   31.600000   31.900000   31.120001   
35         2016-02-24   31.134182   31.820000   31.870001   30.600000   
36         2016-02-25   31.202679   31.889999   32.049999   31.320000   
37         2016-02-26   30.997204   31.680000   32.119999   31.360001   
38         2016-02-29   30.795893   31.360001   32.020000   31.340000   
39         2016-03-01   32.160881   32.750000   32.750000   31.389999   
40         2016-03-02   32.347477   32.939999   33.060001   32.509998   
41         2016-03-03   32.062683   32.650002   33.040001   32.209999   
42         2016-03-04   31.964487   32.549999   32.930000   32.299999   
43         2016-03-07   31.758257   32.340000   32.590000   31.920000   
44         2016-03-08   31.178871   31.750000   32.200001   31.650000   
45         2016-03-09   31.159229   31.730000   32.080002   31.320000   
46         2016-03-10   31.110134   31.680000   32.099998   31.040001   
47         2016-03-11   31.640415   32.220001   32.230000   31.770000   
48         2016-03-14   31.718981   32.299999   32.430000   31.920000   
49         2016-03-15   31.561857   32.139999   32.340000   31.570000   
50         2016-03-16   32.504585   33.099998   33.209999   31.750000   
51         2016-03-17   32.229622   32.820000   33.139999   32.369999   
52         2016-03-18   33.201820   33.810001   34.250000 

# Graphs of the signals

SELL 1, 2 en 3 graphs!

In [193]:
for t in tickers:
    fig, ax = plt.subplots(4, sharex=True, figsize=(9,5), gridspec_kw={'height_ratios':[2.5, 0.5, 0.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)], color='black')
    ax[0].plot(df[('Decycler_d1', t)], color='green', linewidth=1.5)
    ax[0].plot(df[('Decycleru_d1', t)], color='lightgreen', linewidth=1.5)
    ax[0].plot(df[('Decyclerl_d1', t)], color='lightgreen', linewidth=1.5)
    ax[0].plot(df[('SMA1', t)], color='blue', linewidth=1)
    ax[0].plot(df[('Upper_bb1', t)], color='red', linewidth=1)
    ax[0].plot(df[('Mean_bb1', t)], color='red', linewidth=1)
    ax[0].plot(df[('Lower_bb1', t)], color='red', linewidth=1)
    ax[1].plot(df[('Strat_sell1_atr1', t)], color='blue')
    ax[2].plot(df[('Sell2_signal', t)], color='purple')
    ax[3].plot(df[('Sell1_signal', t)], color='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [194]:
for t in tickers:
    fig, ax = plt.subplots(4, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5, 0.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)], color='black')
    ax[1].plot(df[('Buy2_signal', t)], color='blue')
    ax[2].plot(df[('Sell1_signal', t)], color='purple')
    ax[3].plot(df[('Sell2_signal', t)], color='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
for t in tickers:
    fig, ax = plt.subplots(3, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)], color='black')
    ax[0].plot(df[('Stoploss1', t)], color='purple')
    ax[0].plot(df[('Upper_bb1', t)], color='blue')
    ax[0].plot(df[('SMA1', t)], color='green')
    ax[1].plot(df[('Strat_atr1_signal', t)], color='purple')
    ax[2].plot(df[('Strat_decycle_d1_signal', t)], color='red')
    

In [5]:
for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)], color='black')
    ax[0].plot(df[('Stoploss2', t)], color='purple')
    ax[1].plot(df[('Strat_atr2_signal', t)], color='purple')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: ('Strat_atr2_signal', 'NVDA')

In [ ]:
ans = int(input('''Wat weet je?
1.    mg/ml (Milliegram per millieliter)
2.    % (Percentage) 
3.    ‰ (Promille)
Kies: '''))
###Mg/ml
if ans == 1:
    print('Wat is gegeven??')
    mg = int(input('mg (milliegram)'))
    ml = int(input('ml (millieliter)'))
    print('Hoeveel is nodig voor de injectie?')
    nodig = int(input('mg (milliegram)'))
    print('Injecteer')
    x = mg/ml 
    print(f'{nodig/x} ml') 
###Percentage
if ans == 2:
    print('Hoeveel procent (%) is de oplossing?')
    perc = int(input('% (procent)'))
    print('Hoeveel is nodig voor de injectie?')
    nodig = int(input('mg (milliegram)'))
    x = perc*10
    print(f'{nodig/x}')
###Promillage
if ans == 3:
    print('Hoeveel promille ((‰)) is de oplossing')
    prom = int(input('‰ (Promille)'))
    print('Hoeveel is nodig voor de injectie?')
    nodig = int(input('mg (milliegram)'))
    print(f'{nodig/prom}')